In [ ]:
#!import config/Settings.cs

In [ ]:
// Import Semantic Kernel
#r "nuget: Microsoft.SemanticKernel, 1.0.0-beta8"

# 2.1 🚀 Initialization and Configuration

In [ ]:
using Microsoft.SemanticKernel;

var builder = new KernelBuilder();

In [ ]:
using Kernel = Microsoft.SemanticKernel.Kernel;

var (model1, azureEndpoint1, apiKey1, gpt35TurboServiceId) = Settings.LoadFromFile(model: "gpt-35-turbo");
builder.WithAzureOpenAIChatCompletionService(model1, azureEndpoint1, apiKey1, serviceId: gpt35TurboServiceId);

var (model2, azureEndpoint2, apiKey2, gpt4ServiceId) = Settings.LoadFromFile(model: "gpt-4-32k");
builder.WithAzureOpenAIChatCompletionService(model2, azureEndpoint2, apiKey2, serviceId: gpt4ServiceId);

var kernel = builder.Build();

# 2.2. 🚀 Semantic Kernel Pipeline: Calling Chain of Functions

In the SK pipeline, you can create a powerful and expressive chain of inline functions. These functions, when executed sequentially, form a cohesive and efficient processing pipeline. Each function plays a specific role in transforming and analyzing data, contributing to the overall semantic understanding of the code.


In [ ]:
using System;
using Microsoft.SemanticKernel.Orchestration;

SKContext context = kernel.CreateNewContext();

In [ ]:
using Microsoft.SemanticKernel.Connectors.AI.OpenAI;

var requestSettings = new OpenAIRequestSettings
{
    Temperature = 0.7,
    ServiceId = gpt4ServiceId,
};

context.Variables["NumberOfParticipants"] = "3";
context.Variables["NumberOfWords"] = "25";

var chatStoryPrompt = "Craft a {{$NumberOfWords}}-word chat transcript involving {{$NumberOfParticipants}} users engaging with an online virtual assistant.";
var chatStoryFunction = kernel.CreateSemanticFunction(
    promptTemplate: chatStoryPrompt, 
    pluginName: nameof(chatStoryPrompt),
    requestSettings: requestSettings);

In [ ]:
using Microsoft.SemanticKernel.Connectors.AI.OpenAI;

var requestSettings = new OpenAIRequestSettings
{
    Temperature = 2,
    ServiceId = gpt4ServiceId,
};

context.Variables["Lang"] = "Italian";

var translateStoryPrompt = "You are the best linguist. Please translate the given TEXT to {{$Lang}}. TEXT: {{$Input}}";

var translateStoryFunction = kernel.CreateSemanticFunction(
    promptTemplate: translateStoryPrompt, 
    pluginName: nameof(translateStoryPrompt),
    requestSettings: requestSettings);

In [ ]:
ISKFunction[] pipeline = new ISKFunction[]
{
    chatStoryFunction,
    translateStoryFunction,
};

var pipelineResult = await kernel.RunAsync(context.Variables, pipeline);

Console.WriteLine($"RESULT: {string.Join("\n\n===================\n\n", pipelineResult.FunctionResults.Select(_ => _.GetValue<string>()))}");

# 2.3 🚀 Using Built-in SK Plugins: Loading Functions

The SK provides a set of built-in native plugins that offer powerful functionality for code analysis and manipulation. Loading functions from these plugins allows you to leverage their capabilities seamlessly within your code. Follow the steps below to harness the features of built-in native SK plugins.


In [ ]:
using Microsoft.SemanticKernel.Plugins.Core;

var timeFunctions = kernel.ImportFunctions(new TimePlugin(), "time");
context.Variables["input"] = "100";
var result = await kernel.RunAsync(context.Variables, timeFunctions["daysAgo"]);

Console.WriteLine(result);

# 2.4 🚀 Custom SK Plugin: Creating a new Plugin with SK functions

In [ ]:
#!import Plugins/CustomPlugin/CustomPlugin.cs

In [ ]:
// Import your CustomPlugin to Semantic Kernel
var customPluginFunctions = kernel.ImportFunctions(new CustomPlugin(kernel, serviceId: gpt4ServiceId));
Console.WriteLine($"{string.Join("\n", customPluginFunctions.Select(_ => $"[{_.Value.Name}] : {_.Value.Description}"))}");

In [ ]:
// Translate to 'Italian' using 'Translate' function from CustomPlugin
context.Variables["input"] = "Translate me";
context.Variables["lang"] = "Italian";

var result = await kernel.RunAsync(customPluginFunctions["Translate"], context.Variables);

Console.WriteLine($"{result.GetValue<string>()}");

In [ ]:
// Translate to 'Russian' using 'Translate' function from CustomPlugin
context.Variables["input"] = "Translate me";
context.Variables["lang"] = "Russian";

result = await kernel.RunAsync(customPluginFunctions["Translate"], context.Variables);

Console.WriteLine($"{result.GetValue<string>()}");

In [ ]:
// Translate to 'Chinese' using 'Translate' function from CustomPlugin
context.Variables["input"] = "Translate me";
context.Variables["lang"] = "Chinese";

result = await kernel.RunAsync(customPluginFunctions["Translate"], context.Variables);

Console.WriteLine($"{result.GetValue<string>()}");

# 2.5. 🚀 Loading and calling functions from CustomPlugin

In [ ]:
// Create a pipeline array to store custom plugin functions.
var pipeline = new []
{
    customPluginFunctions["Chat"],
    customPluginFunctions["Translate"]
};

context.Variables["lang"] = "Russian";
context.Variables["message"] = "Hello";
context.Variables["history"] = string.Empty;

kernel.ImportFunctions(new TimePlugin());

// Run the pipeline asynchronously using the kernel and update context variables.
var result = await kernel.RunAsync(context.Variables, pipeline);

Console.WriteLine($"[Chat History]: {context.Variables["history"]}");
Console.WriteLine($"[Pipeline Result]: {result.GetValue<string>()}");

In [ ]:
Func<string, Task> Chat = async (string input) => {
    // Save new message in the context variables
    context.Variables["message"] = input;
    var answer = await kernel.RunAsync(context.Variables, pipeline);
};

In [ ]:
await Chat("Favorite color?");

In [ ]:
await Chat("Best movie genre?");

In [ ]:
await Chat("Morning or night?");

# 2.6. 🚀 Invoke a pipelines of semantic functions loaded from various plugins

In [ ]:
// Using plugins from the repo
using Microsoft.SemanticKernel.AI.ChatCompletion;

var currentDirectory = System.IO.Directory.GetCurrentDirectory();
var pluginsDirectory = Path.Combine(currentDirectory, "Plugins");

var summarizePluginFunctions = kernel.ImportSemanticFunctionsFromDirectory(pluginsDirectory, "SummarizePlugin");

In [ ]:
// Creating pipeline with functions from multiple Plugins
var summarizeFunction = summarizePluginFunctions["Summarize"];

var requestSettings = new OpenAIRequestSettings { ServiceId = gpt4ServiceId };
summarizeFunction.ModelSettings.Append(requestSettings);

context.Variables["message"] = "Semantic Kernel or Langchain?";
context.Variables["History"] = string.Empty;
context.Variables["lang"] = "Italian";

var pipeline = new []
{
    customPluginFunctions["Chat"],
    customPluginFunctions["Translate"],
    summarizePluginFunctions["Summarize"],
};

In [ ]:
// Run the pipeline asynchronously using the kernel and update context variables.
var result = await kernel.RunAsync(context.Variables, pipeline);
Console.WriteLine($"[Pipeline Result]: {result.GetValue<string>()}");